In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
from sklearn.preprocessing import LabelEncoder
import os
import joblib

# 1. Introduction

Vấn đề, tầm quan trọng, mục tiêu dự án.

- Vấn đề: Dự đoán giá xe dựa trên các thuộc tính như tuổi xe, loại nhiên liệu, km đã đi, v.v., và phân cụm để hiểu phân khúc thị trường.

- Giải thích tầm quan trọng: Thị trường xe cũ ở Ấn Độ phát triển mạnh (khoảng 3,4 triệu xe giao dịch mỗi năm theo IJERT), ảnh hưởng bởi chính sách (BS-VI), và nhu cầu cá nhân hóa sau COVID-19.

- Đặt mục tiêu: 
    - Xây dựng mô hình Random Forest để dự đoán giá xe chính xác.
    - Khám phá các yếu tố chính ảnh hưởng đến giá.

# 2. Understanding the Data

## 2.1 Load the Data  


In [ ]:
raw_df1 = pd.read_csv('raw_csv/cardekho.csv')
raw_df2 = pd.read_csv('raw_csv/train.csv')
raw_df3 = pd.read_csv('raw_csv/processes2.csv')


## 2.2 Display and Describe Attributes, Number of Records  

In [ ]:
print(raw_df1.shape)
raw_df1.head()

In [ ]:
print(raw_df2.shape)
raw_df2.head()

In [ ]:
print(raw_df3.shape)
raw_df3.head()

# 3. Data Preprocessing

## 3.1 Standardize Column Names, Data Types, and Units  

In [ ]:
# Defind the correct order of columns
correct_order = ['name', 'year', 'selling_price', 'km_driven', 'fuel', 'transmission', 'owner', 'mileage', 'engine', 'max_power', 'seats']

In [ ]:
raw_df1.info()

In [ ]:
raw_df1 = raw_df1[(raw_df1['fuel']!='CNG') & (raw_df1['fuel']!='LPG') & (raw_df1['owner']!='Test Drive Car')]
raw_df1['owner'] = raw_df1['owner'].str.replace(' Owner','',regex=True)
raw_df1['name'] = raw_df1['name'].str.split().str[0]# .str.title()
raw_df1['selling_price'] = raw_df1['selling_price'] / 100000.0

In [ ]:
# Dictionary ánh xạ từ tên cột cũ sang tên cột mới
column_mapping_df1 = {
    "mileage(km/ltr/kg)": "mileage"
}
raw_df1 = raw_df1.drop(columns=['seller_type'])
raw_df1.rename(columns=column_mapping_df1, inplace=True)
print(raw_df1.columns)
raw_df1.head()

In [ ]:
raw_df2.info()

In [ ]:
raw_df2 = raw_df2[raw_df2['Fuel_Type']!='Electric']
raw_df2['Engine'] = raw_df2['Engine'].str.extract('(\d+)')  # Lấy chỉ số trong chuỗi
raw_df2['Power'] = raw_df2['Power'].str.extract('(\d+)')  # Lấy chỉ số trong chuỗi
raw_df2['Mileage'] = raw_df2['Mileage'].str.extract('(\d+)')  # Lấy chỉ số trong chuỗi
raw_df2 = raw_df2[~raw_df2['Mileage'].str.contains('km/kg', na=False)]
raw_df2['Name'] = raw_df2['Name'].str.split(' ').str[0]
raw_df2.head()


In [ ]:
column_mapping_df2 = {
    "Year": "year",
    "Kilometers_Driven": "km_driven",
    "Fuel_Type": "fuel",
    "Transmission": "transmission",
    "Mileage": "mileage",
    "Engine": "engine",
    "Power": "max_power",
    "Seats": "seats",
    "Price": "selling_price",
    "Name": "name",
    "Owner_Type": "owner"
}
raw_df2 = raw_df2.drop(columns=['New_Price', 'Location','Unnamed: 0'])
raw_df2.rename(columns=column_mapping_df2, inplace=True)
raw_df2 = raw_df2[correct_order]
raw_df2.info()
raw_df2.head()

In [ ]:
raw_df3.info()

In [ ]:
raw_df3 = raw_df3[(raw_df3['fuel']!='CNG') & (raw_df3['fuel']!='LPG') & (raw_df3['owner']!='Test Drive Car')]
raw_df3['owner'] = raw_df3['owner'].str.replace(' Owner','',regex=True)
raw_df3['selling_price'] = raw_df3['selling_price'] / 100000.0
print(raw_df3['fuel'].unique())
print(raw_df3['owner'].unique())
raw_df3.head()

In [ ]:
column_mapping_df3 = {
    "max_power (in bph)": "max_power",
    "Mileage": "mileage",
    "Engine (CC)": "engine"
}
raw_df3 = raw_df3.drop(columns=['seller_type', 'Mileage Unit', 'Unnamed: 0'])
raw_df3.rename(columns=column_mapping_df3, inplace=True)
raw_df3 = raw_df3[correct_order]

In [ ]:
def plot_histograms(df, columns, figsize=(15, 12), bins_method='sqrt'):
    """
    Plot histograms for continuous columns in the DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame containing the data.
        columns (list): List of column names to plot histograms for.
        figsize (tuple): Figure size (default is (15, 12)).
        bins_method (str): Method for calculating bins using numpy.histogram_bin_edges() (default is 'sqrt').

    Returns:
        None (Displays the plots).
    """
    plt.figure(figsize=figsize)

    for index, col in enumerate(columns):
        plt.subplot(2, 2, index + 1)
        
        # Convert column to numeric type, removing invalid values
        data = pd.to_numeric(df[col], errors='coerce').dropna()
        
        # Compute bin edges using the specified method
        bin_edges = np.histogram_bin_edges(data, bins=bins_method)
        num_bins = len(bin_edges) - 1  # Actual number of bins
        
        # Plot histogram with the calculated number of bins
        sns.histplot(data=data, bins=num_bins, color=sns.color_palette('pastel')[index], kde=True)
        
        plt.title(col.replace('_', ' ').capitalize(), fontsize=14, pad=10)
        plt.xlabel(col.replace('_', ' ').capitalize(), fontsize=12)
        plt.ylabel('Frequency', fontsize=12)
        
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tick_params(axis='both', labelsize=10)

    plt.tight_layout()
    plt.show()


In [ ]:
continuous_raw_columns = ['km_driven', 'mileage', 'engine', 'max_power']

In [ ]:
raw_df1.info()
raw_df1.describe()

In [ ]:
raw_df2.info()
raw_df2.describe()

In [ ]:
raw_df3.info()
raw_df3.head()

## 3.2 Merge and Save Data

In [ ]:
# Merge 3 datasets
merged_df = pd.concat([raw_df1, raw_df2, raw_df3], ignore_index=True)

# Export to CSV file
saved_file_path = 'raw_csv/merged_dataset.csv'
merged_df.to_csv(saved_file_path, index=False)

# Print message
print(f"Merged and saved to '{saved_file_path}'")

## 3.3 Handle Incorrect and Missing Values

In [ ]:
# Load merge data
merged_df = pd.read_csv(saved_file_path)
print(merged_df.shape)
merged_df.head()

In [ ]:
merged_df.info()
merged_df.describe()

In [ ]:
# Take copy from data to make processes on
preprocessed_df = deepcopy(merged_df)

# Show all types of columns in the data
preprocessed_df.dtypes

In [ ]:
# Check about none values in data to decide if we will make data cleaning or not
preprocessed_df.isnull().sum()

In [ ]:
print("Count of zero values per column:\n", (preprocessed_df == 0).sum())

In [ ]:
# Handling zero values
cols_to_replace_zeros = ['mileage', 'max_power']
preprocessed_df[cols_to_replace_zeros] = preprocessed_df[cols_to_replace_zeros].replace(0, np.nan)

In [ ]:
# Handle missing values of float columns that are mileage, engine, seats
column_float_imputed = ['mileage', 'engine', 'max_power']
preprocessed_df[column_float_imputed] = preprocessed_df[column_float_imputed].fillna(preprocessed_df[column_float_imputed].mean())

# Handle missing values of seats column 
preprocessed_df['seats'] = preprocessed_df['seats'].fillna(preprocessed_df['seats'].mode()[0])


In [ ]:
# Check if there is any missing values in the dataset
print("Count of Null values per column:\n", preprocessed_df.isnull().sum())
print("Count of zero values per column:\n", (preprocessed_df == 0).sum())

## 3.3 Handle textual columns

In [ ]:
# Show all textual columns in the dataset
textual_columns = preprocessed_df.select_dtypes(include = ['object']).columns
preprocessed_df[textual_columns]

In [ ]:
## Label encoding textual columns

# Dictionary to store the encoders
label_encoders = {}

# Fit and transform each textual column, saving the encoders
for col in textual_columns:
    label_encoder = LabelEncoder()
    preprocessed_df[col] = label_encoder.fit_transform(preprocessed_df[col])
    label_encoders[col] = label_encoder

preprocessed_df.head()

In [ ]:
# Check about if there is any extra textual columns
preprocessed_df.select_dtypes(include=['object']).columns

## 3.4 Handle outliers

In [ ]:
def plot_feature_skewness(df, target_column="selling_price"):
    """
    Plots the skewness of numerical features in the dataset.
    
    Parameters:
        df (pd.DataFrame): Preprocessed dataframe.
        target_column (str): The target column to exclude from skewness calculation.
    """
    # Remove target column
    column_skewed = df.columns.drop(target_column)
    
    # Compute skewness
    skewness = df[column_skewed].skew()
    print(skewness)

    # Plot skewness as a bar chart
    plt.figure(figsize=(15, 7))
    sns.barplot(x=column_skewed, y=skewness, hue=column_skewed, palette='coolwarm')

    plt.title('Skewness of Features in Dataset', fontsize=16, pad=20)
    plt.xlabel('Features', fontsize=12)
    plt.ylabel('Skewness', fontsize=12)

    plt.axhline(y=0, color='black', linestyle='--', linewidth=1)
    plt.xticks(rotation=45, ha='right', fontsize=10)

    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

plot_feature_skewness(preprocessed_df)


In [ ]:
# Handle skewness of km driven columns by taking the log function for it
preprocessed_df['km_driven'] = np.log(preprocessed_df['km_driven'] + 1)

# Find skewness for km driven column after we handled it
preprocessed_df['km_driven'].skew()

In [ ]:
plot_feature_skewness(preprocessed_df)

In [ ]:
preprocessed_df.info()
preprocessed_df.head()

## 3.5 Save the preprocessed data and label encoder

In [ ]:
processed_data_dir = "processed_data"
saved_processed_data_path = os.path.join(processed_data_dir, "preprocessed_dataset.csv")
saved_label_encoders_path = os.path.join(processed_data_dir, "label_encoders.sav")
os.makedirs(processed_data_dir, exist_ok=True)

# Save preprocessed DataFrame
preprocessed_df.to_csv(saved_processed_data_path, index=False)

# Save label encoders
joblib.dump(label_encoders, saved_label_encoders_path)

# 4. Statistical Analysis and Model Building on Processed Data

## 4.1 Load the Processed Data

In [ ]:
preprocessed_df = pd.read_csv(saved_processed_data_path)
label_encoders = joblib.load(saved_label_encoders_path)

## 4.2 Distribution of continuous columns

In [ ]:
plot_histograms(preprocessed_df, continuous_raw_columns)

## 4.3 Correlation matrix

In [ ]:
correlation_matrix = preprocessed_df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='Blues', 
            annot_kws={"size": 10}, linewidths=0.5, fmt=".2f")
plt.title('Correlation Matrix', fontsize=20)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
corr_with_price = correlation_matrix['selling_price'].drop('selling_price').sort_values(ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x=corr_with_price.index, y=corr_with_price.values, 
            hue=corr_with_price.index, palette='Blues_d')

plt.title('Correlation of Features with Selling Price', fontsize=16, pad=20)

plt.xlabel('Features', fontsize=12)
plt.ylabel('Correlation Coefficient', fontsize=12)

plt.xticks(rotation=45, ha='right', fontsize=10)

plt.grid(True, axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## 4.4 Overview of categorical features

In [ ]:
categorical_columns = ['name', 'fuel', 'transmission', 'owner', 'seats', 'year']
threshold = 3.0

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

def plot_pie_chart(df, column, ax, encoders):
    data = df[column]

    if column in encoders:
        data = encoders[column].inverse_transform(data)

    value_counts = pd.Series(data).value_counts()
    percentages = value_counts / value_counts.sum() * 100

    mask = percentages < threshold
    if mask.any():
        other_percentage = percentages[mask].sum()
        percentages = percentages[~mask]
        percentages['Other'] = other_percentage

    colors = plt.cm.jet(np.linspace(0, 1, len(percentages)))
    ax.pie(percentages, labels=percentages.index, autopct='%1.1f%%', 
           startangle=90, shadow=True, explode=[0.05] * len(percentages),
           colors=colors)
    ax.axis('equal')
    ax.set_title(f'{column}', fontsize=14)

for i, column in enumerate(categorical_columns):
    if column in preprocessed_df.columns and i < len(axes):
        plot_pie_chart(preprocessed_df, column, axes[i], label_encoders)

plt.tight_layout()
plt.show()

## 4.5 Modeling for car price prediction

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

### 4.5.1 Prepare train and test set

In [ ]:
# Split data into input and label data
X = preprocessed_df.drop(columns = ['selling_price'])
Y = preprocessed_df['selling_price']
print(f'size of input data {X.shape}')
print(f'size of input data {Y.shape}')

In [ ]:
# Split data into train and test data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.12, random_state = 42)
print(f'x train size {x_train.shape}, x test size {x_test.shape}')
print(f'y train size {y_train.shape}, y test size {y_test.shape}')

In [ ]:
# Normalize data (fit on train, transform on test)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)


### 4.5.2 Train models

In [ ]:
# Create all models
models = {
    'Linear Regression': LinearRegression(), 
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'Decision Tree': DecisionTreeRegressor(),
    'KNN': KNeighborsRegressor(),
    'Random Forest': RandomForestRegressor()
}

In [ ]:
results = []
best_model = None
best_model_name = None
best_mse = float('inf')

# Iterate over each model in the models dictionary
for name, model in models.items():
    # Fit the model on the scaled training data
    model.fit(x_train_scaled, y_train)
    
    # Predict on training and testing datasets
    train_pred = model.predict(x_train_scaled)
    test_pred = model.predict(x_test_scaled)
    
    # Calculate evaluation metrics for training data
    train_mse = mean_squared_error(y_train, train_pred)
    train_mae = mean_absolute_error(y_train, train_pred)
    
    # Calculate evaluation metrics for testing data
    test_mse = mean_squared_error(y_test, test_pred)
    test_mae = mean_absolute_error(y_test, test_pred)
    
    # Append all metrics to results list
    results.append({
        'Model': name,
        'Train MSE': train_mse,
        'Test MSE': test_mse,
        'Train MAE': train_mae,
        'Test MAE': test_mae
    })
    
    # Update the best model based on Test MSE
    if test_mse < best_mse:
        best_mse = test_mse
        best_model = model
        best_model_name = name

# Create a DataFrame to display results
df_results = pd.DataFrame(results)
df_results

### 4.5.3 Evaluate and choose the best model

In [ ]:
# Define the metrics and their corresponding DataFrame columns
metrics = [
    ('MSE', 'Train MSE', 'Test MSE'),
    ('MAE', 'Train MAE', 'Test MAE')
]

# Create subplots: one row per metric
fig, axes = plt.subplots(nrows=len(metrics), ncols=1, figsize=(10, 18))

# Loop over each metric to plot its bar chart
for ax, (metric_name, train_col, test_col) in zip(axes, metrics):
    # Set Model as index and select the current metric columns
    df_plot = df_results.set_index('Model')[[train_col, test_col]]
    
    # Plot the bar chart for the current metric
    df_plot.plot(kind='bar', ax=ax, rot=45, title=f"{metric_name} Comparison")
    
    # Add data labels on top of each bar
    for container in ax.containers:
        ax.bar_label(container, fmt='%.2f', padding=3)
    
    # Set the y-axis label
    ax.set_ylabel(metric_name)

plt.tight_layout()
plt.show()


In [ ]:
joblib.dump(best_model, "best_model.pkl")
print(f"Saved {best_model_name} to best_model.pkl")

In [ ]:
if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
    feature_names = X.columns
    
    sorted_idx = np.argsort(importances)[::-1]
    sorted_importances = importances[sorted_idx]
    sorted_features = feature_names[sorted_idx]
    
    plt.figure(figsize=(8, 6))
    plt.barh(sorted_features, sorted_importances, color='skyblue')
    plt.gca().invert_yaxis()
    plt.title(f"Feature Importances - {best_model_name}")
    plt.xlabel("Importance")
    plt.show()
else:
    print(f"The model {best_model_name} does not have the 'feature_importances_' attribute")



In [ ]:
# Load model
loaded_model = joblib.load("best_model.pkl")

# Predict
predictions = loaded_model.predict(x_test_scaled)


In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, predictions, alpha=0.6)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title(f"Actual vs Predicted ({best_model_name})")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_test.values, label="Actual", marker='o', linestyle='dashed')
plt.plot(predictions, label="Predicted", marker='x', linestyle='dotted')
plt.xlabel("Data Points")
plt.ylabel("Price")
plt.title(f"Actual vs Predicted Prices ({best_model_name})")
plt.legend()
plt.show()
